# Доверительные интервалы. Нормальное распределение с неизвестным средним и известной дисперсией

**Задача.** Сравнить на выборках размера 50 для $\mathcal{N}(\theta,4)$ доверительные интервалы:
(1) теоретический, (2) на основе параметрического бутстрэпа, (3) на основе непараметрического бутстрэпа. Сам параметр $\theta$ сгенерировать из равномерного распределения на $[-5,5]$. 

In [ ]:
import numpy as np # для генерации случайных величин и работы с массивами
from scipy import stats # чтобы считать квантили

In [ ]:
np.random.seed(123) # фиксируем seed

In [ ]:
# Фиксируем параметры задачи

n = 50 # размер выборки 
alpha = 0.05 # параметр ошибки

theta = np.random.uniform(-5,5) # неизвестное среднее нормального распределения
sigma = 2 # известная sigma нормального распределения

In [ ]:
# Сгенерируем выборку из нужного распределения
sample = np.random.normal(theta, sigma, size=n)

In [ ]:
print("Значение theta равно",theta)

### Теоретический доверительный интервал

Напомним, что теоретический доверительный интервал вычисляется следующим образом: 

$$
\mathbb{P}\left( \bar{X} - \frac{c_{1-\alpha/2}\sigma}{\sqrt{n}} < \mu < \bar{X} + \frac{c_{1-\alpha/2}\sigma}{\sqrt{n}} \right) = 1-\alpha,
$$
где $c_{\alpha}$ — квантиль распределения $\mathcal{N}(0,1)$ уровня $\alpha$.

In [ ]:
# Вычисляем теоретический доверительный интервал

CI_Theoretical = [np.mean(sample) - stats.norm.ppf(1-alpha/2)*sigma/np.sqrt(n), np.mean(sample) + stats.norm.ppf(1-alpha/2)*sigma/np.sqrt(n)]

In [ ]:
print("Теоретический доверительный интервал равен", CI_Theoretical)

### Доверительный интервал на основе параметрического бутстрэпа

In [ ]:
# Параметры для бутстрэпа
number_of_bootstrap_samples = 10 # количество бутстрэп-выборок
size_of_bootstrap_samples = 20 # размер бутстрэп-выборок

In [ ]:
# Оцениваем неизвестный параметр theta 
mean = np.mean(sample) 

In [ ]:
# Генерируем выборку из распределения N(sample_mean, sigma)
bootstrap_samples = np.random.normal(mean,sigma,size=[number_of_bootstrap_samples,size_of_bootstrap_samples]) 

In [ ]:
# Считаем среднее для каждой выборки 
bootstrap_estimates = np.apply_along_axis(np.mean, 1, bootstrap_samples)

In [ ]:
# Вычисляем параметрический бутстрэп доверительный интервал
CI_Bootstrap_Parametric = [np.quantile(bootstrap_estimates,alpha/2), np.quantile(bootstrap_estimates,1-alpha/2)]

In [ ]:
print("Доверительный интервал на основе парметрического бустрэпа равен", CI_Bootstrap_Parametric)

### Доверительный интервал на основе непараметрического бутстрэпа

In [ ]:
# Будем использовать те же параметры
number_of_bootstrap_samples = 10 # количество бутстрэп-выборок
size_of_bootstrap_samples = 20 # размер бутстрэп-выборок

In [ ]:
# Генерируем выборку из распределения N(bootstrap_mean, sigma)
bootstrap_samples = np.random.choice(sample,size=[number_of_bootstrap_samples,size_of_bootstrap_samples]) 

In [ ]:
# Считаем среднее для каждой выборки 
bootstrap_estimates = np.apply_along_axis(np.mean, 1, bootstrap_samples)

In [ ]:
# Вычисляем непараметрический бутстрэп доверительный интервал
CI_Bootstrap_Nonparametric = [np.quantile(bootstrap_estimates,alpha/2), np.quantile(bootstrap_estimates,1-alpha/2)]

In [ ]:
print("Доверительный интервал на основе (непарметрического) бустрэпа равен", CI_Bootstrap_Nonparametric)

### Как сравнить полученные доверительные интервалы? 

Можно попробовать сравнить длину полученных доверительных интервалов. 
Будет ли длина хорошей оценкой качества интервалов?

In [ ]:
print("Длина теоретического доверительного интервала: ", CI_Theoretical[1]-CI_Theoretical[0])
print("Длина доверительного интервала на основе парметрического бустрэпа: ", CI_Bootstrap_Parametric[1]-CI_Bootstrap_Parametric[0])
print("Длина доверительного интервала на основе непарметрического бустрэпа: ", CI_Bootstrap_Nonparametric[1]-CI_Bootstrap_Nonparametric[0])

Проверим, с какой частотой истинное значение параметра попадает в данные доверительные интервалы

In [ ]:
N_samples = 10000 # количество "экспериентов" по вычислению доверительных интервалов

theoretical = np.zeros(N_samples) # здесь будем хранить результаты для теоретического доверительного интервала
parametric_bootstrap = np.zeros(N_samples) # здесь будем хранить результаты для параметрического бутстрэпа 
nonparametric_bootstrap = np.zeros(N_samples) # здесь будем хранить результаты для непараметрического бутстрэпа 

In [ ]:
# Вычисляем теоретический доверительный интервал

def Theoretical(sample,alpha):
    n = len(sample)
    mean = np.mean(sample)
    return [mean - stats.norm.ppf(1-alpha/2)*sigma/np.sqrt(n), mean + stats.norm.ppf(1-alpha/2)*sigma/np.sqrt(n)]

In [ ]:
# Вычисляем доверительный интервал на основе параметрического бутстрэпа

def Parametric_bootstrap(sample,alpha,number_of_bootstrap_samples,size_of_bootstrap_samples):
    n = len(sample)
    mean = np.mean(sample)
    bootstrap_samples = np.random.normal(mean,sigma,size=[number_of_bootstrap_samples,size_of_bootstrap_samples]) 
    bootstrap_estimates = np.apply_along_axis(np.mean, 1, bootstrap_samples)
    return [np.quantile(bootstrap_estimates,alpha/2), np.quantile(bootstrap_estimates,1-alpha/2)]

In [ ]:
# Вычисляем доверительный интервал на основе непараметрического бутстрэпа

def Nonparametric_bootstrap(sample,alpha,number_of_bootstrap_samples,size_of_bootstrap_samples):
    bootstrap_samples = np.random.choice(sample,size=[number_of_bootstrap_samples,size_of_bootstrap_samples]) 
    bootstrap_estimates = np.apply_along_axis(np.mean, 1, bootstrap_samples)
    return [np.quantile(bootstrap_estimates,alpha/2), np.quantile(bootstrap_estimates,1-alpha/2)]

In [ ]:
# Проведем N_samples экспериментов
for i in range(N_samples):
    sample = np.random.normal(theta, sigma, size=n)
    # теоретические интервалы
    CI_Theoretical = Theoretical(sample,alpha)
    theoretical[i] = (theta >= CI_Theoretical[0]) and (theta <= CI_Theoretical[1])
    
    CI_parametric_bootstrap = Parametric_bootstrap(sample,alpha,number_of_bootstrap_samples,size_of_bootstrap_samples)
    parametric_bootstrap[i] = (theta >= CI_parametric_bootstrap[0]) and (theta <= CI_parametric_bootstrap[1])
    
    CI_nonparametric_bootstrap = Nonparametric_bootstrap(sample,alpha,number_of_bootstrap_samples,size_of_bootstrap_samples)
    nonparametric_bootstrap[i] = (theta >= CI_nonparametric_bootstrap[0]) and (theta <= CI_nonparametric_bootstrap[1])

In [ ]:
print("Частота попадания истинного параметра в доверительный интервал:")
print("- для теоретического доверительного интервала ", np.mean(theoretical))
print("- для параметрического бутстрэп доверительного интервала ", np.mean(parametric_bootstrap))
print("- для непараметрического бутстрэп доверительного интервала ", np.mean(nonparametric_bootstrap))